In [6]:
from langchain_openai import ChatOpenAI

from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, FewShotPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, SimpleJsonOutputParser, PydanticOutputParser
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_classic import hub

# from langchain_openai import OpenAIEmbeddings
from pydantic import BaseModel, Field

llm = ChatOpenAI(
    model="qwen2.5-3b-instruct-q4_k_m.gguf",
    base_url="http://localhost:8002/v1",
    api_key="EMPTY",   # 실제로 사용되지 않음
    temperature=0.2,
)

response = llm.invoke("LangChain이 무엇인지 한 문장으로 설명해줘")
print(response.content)

LangChain은 Chain of Thought를 기반으로 한 인공지능 언어 모델로, 문제 해결 과정을 이해하고 추론하는 능력을 갖추고 있습니다.


In [13]:
from langchain_core.prompts import PromptTemplate
#  PromptTemplate과 ChatPromptTemplate은 동일하게 동작하나, 단일문자열 or 역할기반 메시지의 차이임.

prompt = PromptTemplate(
    template="""
    당신은 AI 튜터입니다.
    아래 질문에 대해 한 문장으로 명확하게 한글로 답하세요.
    
    질문: {question}
    """,
    input_variables=["question"],
)

print(prompt.format(question="LangChain이 왜 필요한가?"))


    당신은 AI 튜터입니다.
    아래 질문에 대해 한 문장으로 명확하게 한글로 답하세요.
    
    질문: LangChain이 왜 필요한가?
    


In [14]:
llm.invoke(prompt.format(question="Ontology가 무엇인가?"))

AIMessage(content='Ontology는 정보의 구조와 의미를 정의하고 관계를 명확히 하는 체계적인 모델 또는 체계입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 68, 'total_tokens': 101, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'qwen2.5-3b-instruct-q4_k_m.gguf', 'system_fingerprint': 'b7654-3333951d8', 'id': 'chatcmpl-tbi8iJCUnySyLWL68p0Y9R7NLmqpZUAp', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9910-2c55-7371-abb8-087b29ff5726-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 68, 'output_tokens': 33, 'total_tokens': 101, 'input_token_details': {}, 'output_token_details': {}})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional translator."),
    ("human", "Translate the following text to Korean:\n{text}")
])

prompt.format_messages(text="Hello world")

[SystemMessage(content='You are a professional translator.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Translate the following text to Korean:\nHello world', additional_kwargs={}, response_metadata={})]

In [7]:
chain = prompt | llm
chain.invoke({"text": "Hello"})

AIMessage(content='Hello is translated to Korean as "안녕하세요" (annyeonghaseyo).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 27, 'total_tokens': 48, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'qwen2.5-3b-instruct-q4_k_m.gguf', 'system_fingerprint': 'b7654-3333951d8', 'id': 'chatcmpl-35YCXEEKqwVgBlEvPOvzHZw5rhFcVcqm', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b990d-dc2d-7a02-98ac-c57413dea261-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 27, 'output_tokens': 21, 'total_tokens': 48, 'input_token_details': {}, 'output_token_details': {}})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리학자입니다."),
    ("user", "{question}")
])

# messages = chat_prompt.invoke(
#     {"question": "LangChain은 무엇인가요?"}
# )

chain = chat_prompt | llm
response = chain.invoke({"question": "독도는 어느 나라 땅 인가요?"})

print(response.content)


독도는 대한민국의 땅입니다. 독도는 경상북도 울릉군에 속해 있으며, 대한민국의 영토입니다.


StrOutputParser

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리학자입니다."),
    ("user", "{question}")
])

# outputparser 추가
output_parser = StrOutputParser()

chain = chat_prompt | llm | output_parser
result = chain.invoke({"question": "독도는 어느 나라 땅 인가요?"})

print(result)       # 출력은 문자열(content 속성 없음)

독도는 대한민국의 땅입니다. 독도는 경상북도 울릉군에 속해 있습니다.


RunnablePassthrough

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리학자입니다."),
    ("user", "{question}")
])

chain = (
    {"question": RunnablePassthrough()}
    | chat_prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke("독도는 어느 나라 땅 인가요?")     # {"question": "..."} 형태로 감싸지 않음 → RunnablePassthrough에서 다음 단계로 그대로 전달
print(result)

독도는 대한민국의 땅입니다.它是韩国的领土.


OutputParser 활용 추가

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

parser = JsonOutputParser()

json_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 정보를 JSON으로만 출력하는 AI입니다."),
    ("user", "질문에 답하세요.\n\n{format_instructions}\n\n질문: {question}")
])

chain = json_prompt | llm | parser 
result = chain.invoke(
    {"question": "독도는 어느 나라 땅 인가요? 국가와 위치, 역사 등을 포함해서 알려주세요.",
     "format_instructions": parser.get_format_instructions()}
)

print(result)

# 결과를 보면 알 수 있듯이, llm이 판단하여 JSON 형식으로 출력을 생성함.

{'location': {'country': '대한민국', 'island_name': '독도', 'coordinates': {'latitude': '36.2500', 'longitude': '126.8000'}}, 'history': {'establishment': '1416년', 'establisher': '조선 왕조', 'significance': '국土의 확장과 통일의象征'}, 'current_status': {'administered_by': '대한민국 정부', 'accessibility': '독도는 현재 일반 관광객의 접근이 제한됩니다.'}}


Pydantic OutputParser (Custom Setting)

In [ ]:
# Pydantic 모델 정의
from pydantic import BaseModel, Field

class Answer(BaseModel):
    summary: str = Field(description="한 문장 요약")
    keywords: list[str] = Field(description="핵심 키워드 목록")


# Parser 생성
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Answer)


# Prompt 구성
prompt = ChatPromptTemplate.from_messages([
    ("system", "출력을 반드시 지정된 형식으로 작성하세요."),
    ("user", "{format_instructions}\n\n질문: {question}")
])

chain = prompt | llm | parser

result = chain.invoke({
    "question": "LangChain의 역할을 설명해줘",
    "format_instructions": parser.get_format_instructions()
})

print(result)
print(type(result))

SimpleJsonOutputParser

In [31]:
from langchain_core.output_parsers import SimpleJsonOutputParser

parser = SimpleJsonOutputParser()

chain = chat_prompt | llm | parser

result = chain.invoke(
    {"question": "AI의 장단점을 JSON으로 정리해줘"}
)

print(result)

{'AI': {'Strengths': [{'Name': 'Speed and Efficiency', 'Description': 'AI can process and analyze large amounts of data much faster than humans.'}, {'Name': 'Accuracy', 'Description': 'AI can perform tasks with high accuracy and consistency, reducing human error.'}, {'Name': 'Automation', 'Description': 'AI can automate repetitive and time-consuming tasks, freeing up human resources.'}, {'Name': 'Insight and Pattern Recognition', 'Description': 'AI can identify patterns and insights in data that might be difficult for humans to detect.'}], 'Weaknesses': [{'Name': 'Lack of Creativity', 'Description': 'AI lacks creativity and cannot generate new ideas or concepts.'}, {'Name': 'Limited Understanding of Context', 'Description': 'AI may struggle with tasks that require understanding of context and human emotions.'}, {'Name': 'Bias and Fairness', 'Description': 'AI systems can be biased if the data they are trained on is biased.'}, {'Name': 'Dependence on Data Quality', 'Description': 'AI sy

In [ ]:
# SimleJsonOutputParser는 스트리밍 방식으로 출력 가능

from langchain_core.output_parsers import SimpleJsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

parser = SimpleJsonOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", "반드시 JSON 형식으로만 응답하세요."),
    ("user", """
질문에 답하세요.

JSON 형식:
{{
  "thought": "...",
  "answer": "..."
}}


질문: {question}
""")
])


### 주의! Prompt에 JSON / dict / code block을 넣을 경우 {}는 항상 {{}}로 쓴다(이스케이프) ###

chain = prompt | llm | parser

for chunk in chain.stream({"question": "독도는 어느 나라 땅 인가요?"}):
    print(chunk)

{}
{'thought': ''}
{'thought': '독'}
{'thought': '독도'}
{'thought': '독도는'}
{'thought': '독도는 대한'}
{'thought': '독도는 대한민'}
{'thought': '독도는 대한민국'}
{'thought': '독도는 대한민국의'}
{'thought': '독도는 대한민국의 땅'}
{'thought': '독도는 대한민국의 땅으로'}
{'thought': '독도는 대한민국의 땅으로,'}
{'thought': '독도는 대한민국의 땅으로, 경'}
{'thought': '독도는 대한민국의 땅으로, 경상'}
{'thought': '독도는 대한민국의 땅으로, 경상북'}
{'thought': '독도는 대한민국의 땅으로, 경상북도'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다.'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다.', 'answer': ''}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다.', 'answer': '독'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다.', 'answer': '독도'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다.', 'answer': '독도는'}
{'thought': '독도는 대한

ChatMessageHistory

In [33]:
# <이전 대화를 포함한 메시지 전달>

# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


# 프롬프트 템플릿 정의: 금융 상담 역할
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다."),
        ("placeholder", "{messages}"),  # 대화 이력 추가
    ]
)

# 프롬프트와 모델을 연결하여 체인 생성
chain = prompt | llm

# 이전 대화를 포함한 메시지 전달
ai_msg = chain.invoke(
    {
        "messages": [
            ("human", "저축을 늘리기 위해 무엇을 할 수 있나요?"),  # 사용자의 첫 질문
            ("ai", "저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요."),  # 챗봇의 답변
            ("human", "방금 뭐라고 했나요?"),  # 사용자의 재확인 질문
        ],
    }
)
print(ai_msg.content)  # 챗봇의 응답 출력

저축을 늘리기 위해 매달 일정 금액을 자동으로 저축하는 것이 좋습니다.


In [34]:
# <`ChatMessageHistory`를 사용한 메시지 관리>

from langchain_community.chat_message_histories import ChatMessageHistory

# 대화 이력 저장을 위한 클래스 초기화
chat_history = ChatMessageHistory()

# 사용자 메시지 추가
chat_history.add_user_message("저축을 늘리기 위해 무엇을 할 수 있나요?")
chat_history.add_ai_message("저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.")

# 새로운 질문 추가 후 다시 체인 실행
chat_history.add_user_message("방금 뭐라고 했나요?")
ai_response = chain.invoke({"messages": chat_history.messages})
print(ai_response.content)  # 챗봇은 이전 메시지를 기억하여 답변합니다.

저축을 늘리기 위해 매달 일정 금액을 자동으로 저축하는 것이 좋습니다.


In [8]:
# <` RunnableWithMessageHistory`를 사용한 메시지 관리>

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# 시스템 메시지와 대화 이력을 사용하는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 모든 질문에 최선을 다해 답변하십시오."),
        ("placeholder", "{chat_history}"),  # 이전 대화 이력
        ("human", "{input}"),  # 사용자의 새로운 질문
    ]
)

# 대화 이력을 관리할 체인 설정
chat_history = ChatMessageHistory()
chain = prompt | llm

# RunnableWithMessageHistory 클래스를 사용해 체인을 감쌉니다
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,  # 세션 ID에 따라 대화 이력을 불러오는 함수
    input_messages_key="input",  # 입력 메시지의 키 설정
    history_messages_key="chat_history",  # 대화 이력의 키 설정
)

# 질문 메시지 체인 실행
chain_with_message_history.invoke(
    {"input": "저축을 늘리기 위해 무엇을 할 수 있나요?"},
    {"configurable": {"session_id": "unused"}},
).content

'저축을 늘리는 방법은 여러 가지가 있습니다. 아래에 몇 가지를 제안해 드릴게요:\n\n1. 예산 관리: 현재 소비 패턴을 분석하고 필요 이상으로 지출하지 않도록 관리하세요. \n\n2. 금융 상품 비교: 다양한 저축 상품을 비교해 보세요. 예를 들어 저축 예금, 저축 상품, 펀드 등 다양한 옵션이 있습니다.\n\n3. 자동 저축 설정: 예금 계좌에 자동으로 저축을 설정하면, 지출이 일어날 때마다 자동으로 저축이 이루어지게 됩니다.\n\n4. 금융 상담: 전문가의 조언을 받는 것도 좋은 방법입니다. 금융 상담사나 투자 전문가와 상담을 통해 올바른 금융 결정을 내릴 수 있습니다.\n\n5. 재무 계획 작성: 목표와 계획을 명확하게 정리하면, 금융 상황을 더 잘 이해하고 관리할 수 있습니다.\n\n6. 저축률 향상: 현재의 저축률을 높이는 것도 중요합니다. 예를 들어, 현재 5%의 저축률이라면 10%로 높이는 것이 좋습니다.\n\n7. 금융 교육: 금융에 대한 이해를 높이는 것도 중요합니다. 다양한 자료를 찾아보거나, 관련 강의나 세미나에 참석하는 것도 도움이 될 수 있습니다.\n\n이러한 방법들을 활용하면 저축률을 높일 수 있을 것입니다.'

In [9]:
# 새로운 입력 메시지를 추가하고 체인을 실행
chain_with_message_history.invoke(
    {"input": "내가 방금 뭐라고 했나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "unused"}}  # 세션 ID 설정
).content

'죄송합니다, 제가前에您刚刚說了什麼?을 이해하지 못했습니다. 다시 말씀해 주시면 감사하겠습니다.'

Trimming

In [10]:
# <메시지 트리밍 예제>

# 라이브러리 불러오기
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# 메시지 트리밍 유틸리티 설정
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

# 트리밍된 대화 이력과 함께 체인 실행
chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter("chat_history") | trimmer)
    | prompt
    | llm
)

# 트리밍된 대화 이력을 사용하는 체인 설정
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# 새로운 대화 내용 추가 후 체인 실행
chain_with_trimmed_history.invoke(
    {"input": "저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "finance_session_1"}}  # 세션 ID 설정
)

AIMessage(content='물론입니다. 5년 내에 집을 사는 계획을 세우는데 필요한 재정 계획을 세우는 것은 매우 중요합니다. 아래에 몇 가지 기본적인 팁을 제공해 드릴게요.\n\n1. **재정 분석**: 현재의 재정 상태를 분석합니다. 월급, 소득, 비용, 저축률 등을 파악합니다. 이를 통해 필요한 금액과 가능한 재정 상태를 파악할 수 있습니다.\n\n2. **목표 설정**: 집을 사는 목표를 명확히 설정합니다. 예를 들어, 특정 지역의 특정 가격대의 집을 사는 것이 목표라면, 이를 명확히 설정해야 합니다.\n\n3. **재정 계획**: 목표를 달성하기 위한 재정 계획을 세웁니다. 예를 들어, 월간 또는 월간 저축률을 설정하고, 이에 따라 필요한 금액을 계산합니다.\n\n4. **투자 계획**: 재정 계획을 위한 투자 계획을 세웁니다. 예를 들어, 주식, 부동산, 펀드 등 다양한 투자 방법을 고려할 수 있습니다.\n\n5. **재정 관리**: 재정 관리를 통해 목표를 달성하는 데 필요한 모든 요소를 관리합니다. 예를 들어, 월간 예산을 설정하고, 필요에 따라 재정을 조절할 수 있습니다.\n\n6. **재정 보험**: 재정 보험을 구매하여 재정 위험을 관리합니다. 예를 들어, 재정 위험을 감소시키는 보험을 구매하거나, 재정 위험을 관리하는 방법을 배우는 것이 좋습니다.\n\n7. **재정 교육**: 재정 관리를 위한 교육을 받습니다. 재정 관리를 위한 다양한 교육 자료나 워크숍 등을 찾아보세요.\n\n이러한 계획을 세우는 것은 쉽지 않을 것입니다. 하지만, 꾸준히 노력하면 5년 후에 집을 사는 것이 가능합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 457, 'prompt_tokens': 112, 'total_tokens': 569, 'completion_tokens_details': None, 'prompt_tokens_d

In [11]:
# 새로운 입력 메시지를 추가하고 체인을 실행
chain_with_trimmed_history.invoke(
    {"input": "내가 방금 뭐라고 했나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "finance_session_1"}}  # 세션 ID 설정
).content

'죄송합니다, 제가前에与えた回答が適切ではありませんでした。5년以内に家を購入するための財政計画を立てることについてアドバイスを提供しました。もう一度質問해 주시면, 더 도움이 될 것 같습니다。'

In [13]:
# <이전 대화 요약 내용 기반으로 답변하기>

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    # 대화를 요약하기 위한 프롬프트 템플릿 설정
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),  # 이전 대화 이력
            (
                "user",
                "이전 대화를 요약해 주세요. 가능한 한 많은 세부 정보를 포함하십시오.",  # 요약 요청 메시지
            ),
        ]
    )

    # 요약 체인 생성 및 실행
    summarization_chain = summarization_prompt | llm
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})
    chat_history.clear()  # 요약 후 이전 대화 삭제
    chat_history.add_message(summary_message)  # 요약된 메시지를 대화 이력에 추가
    return True

In [14]:
# 대화 요약을 처리하는 체인 설정
chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

# 요약된 대화를 기반으로 새로운 질문에 응답
print(chain_with_summarization.invoke(
    {"input": "저에게 어떤 재정적 조언을 해주셨나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "unused"}}  # 세션 ID 설정
).content)

죄송합니다, 제가 잘못 이해했습니다. 재정적 조언에 대해 구체적으로 질문해 주셨다면, 그 질문에 답변 드리겠습니다. 예를 들어, 집을 사는 목표를 가지고 계획을 세우는 방법, 월간 저축률 계산 방법, 재정 위험 관리 방법 등에 대한 질문이 있으면 답변 드릴 수 있습니다. 어떤 재정적 상황에 대한 조언이 필요하신지 좀 더 구체적으로 알려주시면 감사하겠습니다.


Few Shot Prompt

In [5]:
# <`PromptTemplate`를 이용한 퓨샷 프롬프트>

# 라이브러리 불러오기
from langchain_core.prompts import PromptTemplate
# 질문과 답변을 포맷하는 프롬프트 템플릿 정의
example_prompt = PromptTemplate.from_template("질문: {question}\n답변: {answer}")

examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자
""",
    } ,
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
""",
    },
]

print(example_prompt.invoke(examples[0]).to_string())

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자



In [7]:
# <`FewShotPromptTemplate` 를 이용한 퓨샷 프롬프트>

# 라이브러리 불러오기
from langchain_core.prompts import FewShotPromptTemplate
# FewShotPromptTemplate 생성
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="질문: {input}",
    input_variables=["input"],
)
# '부동산 투자' 주제로 프롬프트 호출 및 출력
print(
    prompt.invoke({"input": "부동산 투자의 장점은 무엇인가?"}).to_string()
)

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자


질문: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권


질문: 부동산 투자의 장점은 무엇인가?


In [8]:
# <예제 선택기를 이용한 퓨샷 프롬프트>

# 라이브러리 불러오기
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# # 예제 선택기 초기화
# example_selector = SemanticSimilarityExampleSelector.from_examples(
#     examples,  # 사용할 예제 목록
#     OpenAIEmbeddings(api_key=api_key),  # 임베딩 생성에 사용되는 클래스
#     Chroma,  # 임베딩을 저장하고 유사도 검색을 수행하는 벡터 스토어 클래스
#     k=1,  # 선택할 예제의 수
# )


embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,      # 사용할 예제 목록
    embeddings,    # 임베딩 생성에 사용되는 클래스
    Chroma,        # 임베딩을 저장하고 유사도 검색을 수행하는 벡터 스토어 클래스
    k=1,           # 선택할 예제의 수
)


# 입력과 가장 유사한 예제 선택
question = "부동산 투자의 장점은 무엇인가?"
selected_examples = example_selector.select_examples({"question": question})

# 선택된 예제 출력
print(f"입력과 가장 유사한 예제: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

C:\Users\wind9\AppData\Local\Temp\ipykernel_22324\3617487609.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

d:\llm_serving\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\llm_serving\llm_servingmodels\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

입력과 가장 유사한 예제: 부동산 투자의 장점은 무엇인가?


question: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
answer: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권



In [ ]:
# <퓨샷 프롬프트 AI 모델 적용>

# 라이브러리 불러오기
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

# 예제 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="질문: {question}\n답변: {answer}"
)

# 퓨샷 프롬프트 템플릿 설정
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    
    prefix="다음은 금융 관련 질문과 답변의 예시입니다:",
    suffix="질문: {input}\n답변:",
    input_variables=["input"]
)

In [12]:
# AI 모델 설정
# model = ChatOpenAI(model_name="gpt-4o")


# 체인 구성 및 실행
# chain = prompt | model  # RunnableSequence를 사용하여 체인 연결

chain = prompt | llm

response = chain.invoke({"input": "부동산 투자의 장점은 무엇인가?"})  # invoke 메서드 사용
print(response.content)

부동산 투자의 장점은 다음과 같습니다:

1. **재산 증가**: 부동산은 일반적으로 시간이 지날수록 가치가 증가합니다.
2. **재무적 안정성**: 부동산은 장기적으로 보유하면 매입 가격과 비교해 매각 가격이 상승하는 경향이 있습니다.
3. **수익 창출**: 부동산은 임대료를 통해 수익을 얻을 수 있습니다.
4. **투자 다양화**: 부동산 투자는 다른 투자 형태와 함께 다양화된 투자를 할 수 있습니다.
5. **재무레버리지 활용**: 부동산 투자는 장기적으로 보유하면 재무레버리지를 활용하여 더 큰 수익을 얻을 수 있습니다.

하지만 부동산 투자는 부동산 가격 변동성, 관리 비용, 세금 부담 등 다양한 위험 요소가 존재합니다. 따라서 투자 전에는 충분한 조사를 통해 적절한 투자 결정을 내릴 필요가 있습니다.


In [13]:
# <랭체인 허브의 특정 프롬프트 불러오기>
# pip install langchain-classic
from langchain_classic import hub

# 최신 버전의 프롬프트 불러오기
prompt = hub.pull("hardkothari/prompt-maker")
# 특정 버전의 프롬프트 불러오기
hub.pull("hardkothari/prompt-maker")

ChatPromptTemplate(input_variables=['lazy_prompt', 'task'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hardkothari', 'lc_hub_repo': 'prompt-maker', 'lc_hub_commit_hash': 'c5db8eeefa7be4862a9599b759608dd10ee53f53910838f69abb5ab31c257c2d'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert Prompt Writer for Large Language Models.\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lazy_prompt', 'task'], input_types={}, partial_variables={}, template='Your goal is to improve the prompt given below for {task} :\n--------------------\n\nPrompt: {lazy_prompt}\n\n--------------------\n\nHere are several tips on writing great prompts:\n\n-------\n\nStart the prompt by stating that it is an expert in the subject.\n\nPut instructions at the beginning of the prompt and use ### or to separate the instruction and context \n\nBe specific, 